# 11) Paper 2 — IFRS9 End-to-End

Notebook de soporte para el paper IFRS9:
- **Objetivo**: consolidar evidencia de escenarios ECL, sensibilidad y staging.
- **Salidas**: `reports/paper_material/paper2/figures/` y `reports/paper_material/paper2/tables/`.

In [ ]:
from __future__ import annotations

import json
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"

PROJECT_ROOT = (
    Path.cwd().resolve().parent if Path.cwd().name == "notebooks" else Path.cwd().resolve()
)
DATA_DIR = PROJECT_ROOT / "data" / "processed"
MODEL_DIR = PROJECT_ROOT / "models"


def load_parquet(name: str) -> pd.DataFrame:
    return pd.read_parquet(DATA_DIR / f"{name}.parquet")


def load_json(name: str, from_models: bool = False) -> dict:
    base = MODEL_DIR if from_models else DATA_DIR
    return json.loads((base / f"{name}.json").read_text())


def ensure_dirs(base: Path) -> dict[str, Path]:
    dirs = {
        "base": base,
        "fig": base / "figures",
        "tbl": base / "tables",
    }
    for d in dirs.values():
        d.mkdir(parents=True, exist_ok=True)
    return dirs


def export_figure(fig: go.Figure, stem: str, out_fig_dir: Path) -> None:
    html_path = out_fig_dir / f"{stem}.html"
    fig.write_html(html_path)
    try:
        png_path = out_fig_dir / f"{stem}.png"
        fig.write_image(png_path, width=1400, height=850, scale=2)
        print(f"Saved: {html_path} and {png_path}")
    except Exception as exc:  # noqa: BLE001
        print(f"Saved HTML only ({html_path}). PNG skipped: {exc}")


def export_table(df: pd.DataFrame, stem: str, out_tbl_dir: Path, max_rows: int = 2000) -> None:
    csv_path = out_tbl_dir / f"{stem}.csv"
    tex_path = out_tbl_dir / f"{stem}.tex"
    out_df = df.copy().head(max_rows)
    out_df.to_csv(csv_path, index=False)
    try:
        latex = out_df.to_latex(index=False, escape=False)
        tex_path.write_text(latex, encoding="utf-8")
        print(f"Saved: {csv_path} and {tex_path}")
    except Exception as exc:  # noqa: BLE001
        print(f"Saved CSV only ({csv_path}). LaTeX skipped: {exc}")

In [ ]:
out = ensure_dirs(PROJECT_ROOT / "reports" / "paper_material" / "paper2")
pipeline_summary = load_json("pipeline_summary")
ifrs9_summary = load_parquet("ifrs9_scenario_summary")
ifrs9_grid = load_parquet("ifrs9_sensitivity_grid")
ifrs9_grade = load_parquet("ifrs9_scenario_grade_summary")
print("ifrs9_summary", ifrs9_summary.shape)
print("ifrs9_grid", ifrs9_grid.shape)
print("ifrs9_grade", ifrs9_grade.shape)

In [ ]:
pipeline = pipeline_summary.get("pipeline", {})
stages = pipeline.get("stages", {}) if isinstance(pipeline.get("stages"), dict) else {}
baseline = ifrs9_summary.loc[ifrs9_summary["scenario"] == "baseline"]
severe = ifrs9_summary.loc[ifrs9_summary["scenario"] == "severe_stress"]
baseline_ecl = float(baseline["total_ecl"].iloc[0]) if not baseline.empty else np.nan
severe_ecl = float(severe["total_ecl"].iloc[0]) if not severe.empty else np.nan
uplift = (
    (severe_ecl / baseline_ecl - 1.0)
    if np.isfinite(baseline_ecl) and baseline_ecl > 0 and np.isfinite(severe_ecl)
    else np.nan
)
key = pd.DataFrame(
    [
        {"metric": "stage1_n", "value": stages.get("S1", np.nan)},
        {"metric": "stage2_n", "value": stages.get("S2", np.nan)},
        {"metric": "stage3_n", "value": stages.get("S3", np.nan)},
        {"metric": "baseline_ecl", "value": baseline_ecl},
        {"metric": "severe_ecl", "value": severe_ecl},
        {"metric": "severe_uplift", "value": uplift},
    ]
)
key

In [ ]:
# Figure 1: stage composition by scenario
stage_df = ifrs9_summary[["scenario", "stage1_share", "stage2_share", "stage3_share"]].melt(
    id_vars=["scenario"],
    var_name="stage",
    value_name="share",
)
fig1 = px.bar(
    stage_df,
    x="scenario",
    y="share",
    color="stage",
    barmode="stack",
    title="Paper2-Fig1: Stage Composition by Scenario",
)
fig1
export_figure(fig1, "paper2_fig1_stage_composition", out["fig"])

In [ ]:
# Figure 2: ECL range by scenario
range_df = ifrs9_summary[["scenario", "total_ecl_low", "total_ecl_point", "total_ecl_high"]].melt(
    id_vars=["scenario"],
    var_name="estimate",
    value_name="ecl",
)
fig2 = px.bar(
    range_df,
    x="scenario",
    y="ecl",
    color="estimate",
    barmode="group",
    title="Paper2-Fig2: ECL Low/Point/High by Scenario",
)
fig2
export_figure(fig2, "paper2_fig2_ecl_range_by_scenario", out["fig"])

In [ ]:
# Figure 3: sensitivity heatmap
disc = (
    0.05
    if 0.05 in ifrs9_grid["discount_rate"].unique()
    else float(ifrs9_grid["discount_rate"].iloc[0])
)
slice_df = ifrs9_grid.loc[ifrs9_grid["discount_rate"] == disc]
heat = (
    slice_df.pivot_table(index="pd_mult", columns="lgd_mult", values="total_ecl", aggfunc="mean")
    / 1_000_000
)
fig3 = px.imshow(
    heat,
    text_auto=".1f",
    labels={"x": "LGD multiplier", "y": "PD multiplier", "color": "ECL (MM)"},
    title=f"Paper2-Fig3: ECL Sensitivity Heatmap (discount={disc:.2f})",
)
fig3
export_figure(fig3, "paper2_fig3_ecl_sensitivity_heatmap", out["fig"])

In [ ]:
# Export tables
export_table(key, "paper2_table0_key_metrics", out["tbl"])
export_table(ifrs9_summary, "paper2_table1_scenario_summary", out["tbl"])
export_table(ifrs9_grid, "paper2_table2_sensitivity_grid", out["tbl"], max_rows=5000)
export_table(ifrs9_grade, "paper2_tableA1_grade_summary", out["tbl"])

## Threats to Validity (draft)
- Trigger SICR por incertidumbre requiere calibración con política institucional.
- Escenarios macro por multiplicadores son simplificados frente a escenarios oficiales.

## Reproducibilidad
```bash
uv run dvc repro run_ifrs9_sensitivity build_pipeline_results export_streamlit_artifacts
uv run pytest -q tests/test_evaluation/test_ifrs9.py
```